In [1]:
import numpy as np
import pandas as pd
import os
import DataProvider

In [2]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [3]:
def forward_propagate(X, theta1, theta2):
    m = X.shape[0]
    
    a1 = np.insert(X, 0, values=np.ones(m), axis=1)
    z2 = a1 * theta1.T
    a2 = np.insert(sigmoid(z2), 0, values=np.ones(m), axis=1)
    z3 = a2 * theta2.T
    h = sigmoid(z3)
    
    return a1, z2, a2, z3, h

In [4]:
def cost(params, input_size, hidden_size, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)
    
    # reshape the parameter array into parameter matrices for each layer
    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))
    
    # run the feed-forward pass
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
    
    # compute the cost
    J = 0
    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        J += np.sum(first_term - second_term)
    
    J = J / m
    
    # add the cost regularization term
    J += (float(learning_rate) / (2 * m)) * (np.sum(np.power(theta1[:,1:], 2)) + np.sum(np.power(theta2[:,1:], 2)))
    
    return J

In [5]:
def sigmoid_gradient(z):
    return np.multiply(sigmoid(z), (1 - sigmoid(z)))

In [6]:
def backprop(params, input_size, hidden_size, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)
    
    # reshape the parameter array into parameter matrices for each layer
    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))
    
    # run the feed-forward pass
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
    
    # initializations
    J = 0
    delta1 = np.zeros(theta1.shape)  # (25, 401)
    delta2 = np.zeros(theta2.shape)  # (10, 26)
    
    # compute the cost
    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        J += np.sum(first_term - second_term)
    
    #print(J)
    J = J / m
    
    # add the cost regularization term
    J += (float(learning_rate) / (2 * m)) * (np.sum(np.power(theta1[:,1:], 2)) + np.sum(np.power(theta2[:,1:], 2)))
    
    # perform backpropagation
    for t in range(m):
        a1t = a1[t,:]  # (1, 401)
        z2t = z2[t,:]  # (1, 25)
        a2t = a2[t,:]  # (1, 26)
        ht = h[t,:]  # (1, 10)
        yt = y[t,:]  # (1, 10)
        
        d3t = ht - yt  # (1, 10)
        
        z2t = np.insert(z2t, 0, values=np.ones(1))  # (1, 26)
        d2t = np.multiply((theta2.T * d3t.T).T, sigmoid_gradient(z2t))  # (1, 26)
        
        delta1 = delta1 + (d2t[:,1:]).T * a1t
        delta2 = delta2 + d3t.T * a2t
        
    delta1 = delta1 / m
    delta2 = delta2 / m
    
    # add the gradient regularization term
    delta1[:,1:] = delta1[:,1:] + (theta1[:,1:] * learning_rate) / m
    delta2[:,1:] = delta2[:,1:] + (theta2[:,1:] * learning_rate) / m
    
    # unravel the gradient matrices into a single array
    grad = np.concatenate((np.ravel(delta1), np.ravel(delta2)))
    
    return J, grad

In [7]:
# pre_initial setup
prov = DataProvider.DataProvider()

# initial setup
input_size = prov.returnSizeOfEverySpectogram()
hidden_size = 5
num_labels = 5
learning_rate = 1

# randomly initialize a parameter array of the size of the full network's parameters
params = (np.random.random(size=hidden_size * (input_size + 1) + num_labels * (hidden_size + 1)) - 0.5) * 0.25
#print(params.shape)
#params = np.ones(hidden_size * (input_size + 1) + num_labels * (hidden_size + 1))

# unravel the parameter array into parameter matrices for each layer
theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

theta1.shape, theta2.shape

[new value of  15
--new value of  16
---new value of  63
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

/home/rezi/Desktop/Machine Learning/ML_Final_project/DataProvider.py:135: RuntimeWarning: invalid value encountered in true_divide
  spectrogram = (spectrogram - mean) / std


------------------------------------------------------------------------------------------------------------------------new value of  72
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------]
[##############################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################

((5, 1441), (5, 6))

In [8]:
X = list()
y = list()
trainVal = list()

In [9]:
while prov.hasNext(True):
    Xt, yt = prov.next(True)
    for i in range(len(yt)):
        if yt[i] == 1:
            trainVal.append(i+1)
            
    X.append(Xt)
    y.append(yt)

In [10]:
X = np.nan_to_num(np.matrix(X))
y = np.matrix(y)

X.shape, y.shape

((688, 1440), (688, 5))

In [12]:
for index in range(len(X)):
    Xt = X[index]
    yt = y[index]
    J, grad = backprop(params, input_size, hidden_size, num_labels, Xt, yt, learning_rate)
    params -= grad

In [13]:
params

array([-2.13096057, -0.00368179,  0.01238031, ..., -0.00254352,
       -0.00256396, -0.00256873])

In [14]:
X2 = list()
y2 = list()
actualVal = list()

In [15]:
while prov.hasNext(False):
    X2t, y2t = prov.next(False)
    actualVal.append(y2t)
    
    k = np.zeros(5)
    k[y2t - 1] = 1
    
    X2.append(X2t)
    y2.append(k)

In [16]:
X2 = np.nan_to_num(np.matrix(X2))
y2 = np.matrix(y2)

X2.shape, y2.shape

((8, 1440), (8, 5))

In [17]:
testRes = []

theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

In [19]:
for index in range(len(X)):
    X2t = X2[index]
    y2t = y2[index]
    print(y2t)
    a1, z2, a2, z3, h = forward_propagate(X2t, theta1, theta2)
    
    testRes.append((np.sum(h, axis = 0), y2t))

[[0. 0. 1. 0. 0.]]
[[0. 1. 0. 0. 0.]]
[[0. 1. 0. 0. 0.]]
[[0. 0. 0. 0. 1.]]
[[0. 0. 0. 0. 1.]]
[[0. 0. 1. 0. 0.]]
[[1. 0. 0. 0. 0.]]
[[0. 0. 0. 1. 0.]]


IndexError: index 8 is out of bounds for axis 0 with size 8

In [20]:
testRes

[(matrix([[0.04437996, 0.58438951, 0.02048243, 0.41334473, 0.02530747]]),
  matrix([[0., 0., 1., 0., 0.]])),
 (matrix([[0.04434555, 0.58672997, 0.02047509, 0.41102109, 0.02529627]]),
  matrix([[0., 1., 0., 0., 0.]])),
 (matrix([[0.04433924, 0.58715915, 0.02047374, 0.41059501, 0.02529421]]),
  matrix([[0., 1., 0., 0., 0.]])),
 (matrix([[0.04438507, 0.58404202, 0.02048352, 0.41368974, 0.02530913]]),
  matrix([[0., 0., 0., 0., 1.]])),
 (matrix([[0.04438728, 0.58389154, 0.02048399, 0.41383913, 0.02530985]]),
  matrix([[0., 0., 0., 0., 1.]])),
 (matrix([[0.04438244, 0.58422139, 0.02048296, 0.41351164, 0.02530827]]),
  matrix([[0., 0., 1., 0., 0.]])),
 (matrix([[0.04436709, 0.58526502, 0.02047969, 0.41247551, 0.02530328]]),
  matrix([[1., 0., 0., 0., 0.]])),
 (matrix([[0.04437809, 0.58451728, 0.02048203, 0.41321788, 0.02530686]]),
  matrix([[0., 0., 0., 1., 0.]]))]